In [2]:
# When comparing two data sets using statistical tools, it is not
# sufficient to simply compare their average or median values. This is because
# such results can be skewed by randomness. For example, suppose we are
# calculating the median values of ten seeds. If one algorithm gets "lucky"
# and happens to use more above-average seeds, the estimated median will
# be skewed. Therefore, it is necessary to check the statistical significance of
# results. This is exactly what the indifferent entries are displaying. To determine
# statistical significance, the Kruskal-Wallis and Mann-Whitney U tests with 95% confidence 
# intervals can be used.

# The Mann-Whitney U test is a nonparametric test of the null hypothesis that the distribution 
# underlying sample x is the same as the distribution underlying sample y. 
# It is often used as a test of difference in location between distributions.

In [3]:
from scipy import stats
import sys
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 8]
sys.path.append('..')
import rbf_functions

In [4]:
rbfs = [rbf_functions.original_rbf,
        rbf_functions.squared_exponential_rbf,
        rbf_functions.inverse_quadratic_rbf,
        rbf_functions.inverse_multiquadric_rbf,
        rbf_functions.exponential_rbf,
        rbf_functions.matern32_rbf,
        rbf_functions.matern52_rbf,
       ]

# Mann-Whitney HV local refset

In [45]:
gddata = {}
eidata = {}
hvdata = {}
varlist = ['10', '20', '30', '40', '50', '60', '70', '80', '90', '100']
for entry in rbfs:
    rbf = entry.__name__
#     df_gd_data = pd.read_csv(f"gd_all/{rbf}_gd_all.csv")
#     df_ei_data = pd.read_csv(f"ei_all/{rbf}_ei_all.csv")
    df_hv_data = pd.read_csv(f"hv/{rbf}_hv.csv")
    gddata[rbf] = {}
    eidata[rbf] = {}
    hvdata[rbf] = {}
    for i in varlist: # add max hv value at 100.000 nfe to dict for each seed
#         gddata[rbf][i] = df_gd_data[i].dropna().iloc[-1] 
#         eidata[rbf][i] = df_ei_data[i].dropna().iloc[-1]
        hvdata[rbf][i] = df_hv_data[i].dropna().iloc[-1]
# df_gd = pd.DataFrame.from_dict(gddata, orient='index')
# df_ei = pd.DataFrame.from_dict(eidata, orient='index')
df_hv = pd.DataFrame.from_dict(hvdata, orient='index')

In [46]:
# Max attained hypervolume per rbf for each seed
df_hv

,10,20,30,40,50,60,70,80,90,100
original_rbf,0.465507,0.392343,0.476701,0.523039,0.426644,0.428990,0.439437,0.501185,0.491427,0.515935
squared_exponential_rbf,0.380046,0.410288,0.322934,0.467233,0.435752,0.422746,0.537120,0.464505,0.352284,0.503771
inverse_quadratic_rbf,0.393558,0.362230,0.367310,0.401038,0.368133,0.361958,0.389456,0.368153,0.356869,0.392215
inverse_multiquadric_rbf,0.483241,0.454009,0.469192,0.472602,0.480215,0.480522,0.474922,0.471801,0.477761,0.464382
exponential_rbf,0.226676,0.316662,0.344283,0.351767,0.235409,0.356604,0.242845,0.340627,0.222944,0.275830
matern32_rbf,0.400585,0.432749,0.462341,0.329929,0.266519,0.428962,0.448612,0.281948,0.401901,0.498354
matern52_rbf,0.326209,0.324530,0.445316,0.370571,0.284946,0.363744,0.332511,0.425792,0.275986,0.426712


In [ ]:
# method:
# 'asymptotic': compares the standardized test statistic against the normal distribution, correcting for ties.
# 'exact': computes the exact p-value by comparing the observed  statistic against the exact distribution of the  statistic under the null hypothesis. No correction is made for ties.
# 'auto': chooses 'exact' when the size of one of the samples is less than 8 and there are no ties; chooses 'asymptotic' otherwise.

In [47]:
df_mannwhitney = pd.DataFrame(columns=df_hv.index)
df_pval = pd.DataFrame(columns=df_hv.index)
for i, row in df_hv.iterrows():
    for j, row2 in df_hv.iterrows():
        mannwhit, pval = stats.mannwhitneyu(row, row2, method='auto')
        df_mannwhitney.loc[i, j] = mannwhit
        df_pval.loc[i, j] = pval

In [48]:
df_mannwhitney

,original_rbf,squared_exponential_rbf,inverse_quadratic_rbf,inverse_multiquadric_rbf,exponential_rbf,matern32_rbf,matern52_rbf
original_rbf,50.0,68.0,98.0,48.0,100.0,78.0,93.0
squared_exponential_rbf,32.0,50.0,76.0,24.0,95.0,63.0,77.0
inverse_quadratic_rbf,2.0,24.0,50.0,0.0,100.0,31.0,61.0
inverse_multiquadric_rbf,52.0,76.0,100.0,50.0,100.0,89.0,100.0
exponential_rbf,0.0,5.0,0.0,0.0,50.0,15.0,22.0
matern32_rbf,22.0,37.0,69.0,11.0,85.0,50.0,67.0
matern52_rbf,7.0,23.0,39.0,0.0,78.0,33.0,50.0


In [20]:
df_pval

,original_rbf,squared_exponential_rbf,inverse_quadratic_rbf,inverse_multiquadric_rbf,exponential_rbf,matern32_rbf,matern52_rbf
original_rbf,1.0,0.185877,0.00033,0.909722,0.000183,0.037635,0.001315
squared_exponential_rbf,0.185877,1.0,0.053903,0.053903,0.000769,0.344704,0.045155
inverse_quadratic_rbf,0.00033,0.053903,1.0,0.000183,0.000183,0.161972,0.427355
inverse_multiquadric_rbf,0.909722,0.053903,0.000183,1.0,0.000183,0.003611,0.000183
exponential_rbf,0.000183,0.000769,0.000183,0.000183,1.0,0.009108,0.037635
matern32_rbf,0.037635,0.344704,0.161972,0.003611,0.009108,1.0,0.212294
matern52_rbf,0.001315,0.045155,0.427355,0.000183,0.037635,0.212294,1.0


# Mann-Whitney  on objectives

In [5]:
rbfd = {}
rbfn = []
for rbf in rbfs:
    name = rbf.__name__
    rbfn.append(name)
    rbfd[name] = pd.read_csv(f"refsets/{name}_refset.csv")

In [ ]:
# The lower the U is, the more different the groups are.
# https://www.youtube.com/watch?v=nRAAAp1Bgnw
# https://www.youtube.com/watch?v=Twk6lBhBl88

# Hydropower

In [22]:
df_mannwhit_hyd = pd.DataFrame(columns=rbfn)
df_pval_hyd = pd.DataFrame(columns=rbfn)
for i in rbfd:
    for j in rbfd:
        row1 = rbfd[i]['hydropower']
        row2 = rbfd[j]['hydropower']
        mannwhit, pval = stats.mannwhitneyu(row1, row2, method='auto')
        df_mannwhit_hyd.loc[i, j] = mannwhit
        df_pval_hyd.loc[i, j] = pval
        
df_mannwhit_hyd

,original_rbf,squared_exponential_rbf,inverse_quadratic_rbf,inverse_multiquadric_rbf,exponential_rbf,matern32_rbf,matern52_rbf
original_rbf,1146098.0,1007360.0,1381423.0,1149758.0,1198040.0,1395145.0,836346.0
squared_exponential_rbf,1047138.0,920724.5,1265240.0,1048806.0,1116918.0,1341540.0,811117.0
inverse_quadratic_rbf,535301.0,452722.0,801378.0,690426.0,545703.0,529321.0,325569.0
inverse_multiquadric_rbf,601940.0,521243.0,774336.0,669324.5,534664.0,586255.0,371258.0
exponential_rbf,234204.0,166804.0,651933.0,559858.0,447458.0,252682.0,126979.0
matern32_rbf,563971.0,414418.0,1108883.0,910903.0,971442.0,837218.0,487662.0
matern52_rbf,461152.0,351832.0,759393.0,620291.0,683743.0,621296.0,367224.5


In [9]:
df_pval_hyd

,original_rbf,squared_exponential_rbf,inverse_quadratic_rbf,inverse_multiquadric_rbf,exponential_rbf,matern32_rbf,matern52_rbf
original_rbf,1.0,0.369768,0.0,0.0,0.0,0.0,0.0
squared_exponential_rbf,0.369768,1.0,0.0,0.0,0.0,0.0,0.0
inverse_quadratic_rbf,0.0,0.0,1.0,0.014726,0.000352,0.0,0.0
inverse_multiquadric_rbf,0.0,0.0,0.014726,1.0,0.363194,0.0,0.0
exponential_rbf,0.0,0.0,0.000352,0.363194,1.0,0.0,0.0
matern32_rbf,0.0,0.0,0.0,0.0,0.0,1.0,0.000002
matern52_rbf,0.0,0.0,0.0,0.0,0.0,0.000002,1.0


# Atomic power plant

In [10]:
df_mannwhit_atom = pd.DataFrame(columns=rbfn)
df_pval_atom = pd.DataFrame(columns=rbfn)
for i in rbfd:
    for j in rbfd:
        row1 = rbfd[i]['atomicpowerplant']
        row2 = rbfd[j]['atomicpowerplant']
        mannwhit, pval = stats.mannwhitneyu(row1, row2, method='auto')
        df_mannwhit_atom.loc[i, j] = mannwhit
        df_pval_atom.loc[i, j] = pval
        
df_mannwhit_atom

,original_rbf,squared_exponential_rbf,inverse_quadratic_rbf,inverse_multiquadric_rbf,exponential_rbf,matern32_rbf,matern52_rbf
original_rbf,1146098.0,1219750.0,1090500.0,966971.0,1151946.0,1191100.0,773385.0
squared_exponential_rbf,834748.0,920724.5,803197.0,730027.0,863022.0,880128.0,556626.0
inverse_quadratic_rbf,826224.0,914765.0,801378.0,723611.0,873093.0,876541.0,557166.0
inverse_multiquadric_rbf,784727.0,840022.0,741151.0,669324.5,770844.0,812724.0,521013.0
exponential_rbf,280298.0,420700.0,324543.0,323678.0,447458.0,351825.0,190590.0
matern32_rbf,768016.0,875830.0,761663.0,684434.0,872299.0,837218.0,533889.0
matern52_rbf,524113.0,606323.0,527796.0,470536.0,620132.0,575069.0,367224.5


In [11]:
df_pval_atom

,original_rbf,squared_exponential_rbf,inverse_quadratic_rbf,inverse_multiquadric_rbf,exponential_rbf,matern32_rbf,matern52_rbf
original_rbf,1.0,0.0,0.0,0.000004,0.0,0.0,0.0
squared_exponential_rbf,0.0,1.0,0.004001,0.002431,0.0,0.913152,0.089894
inverse_quadratic_rbf,0.0,0.004001,1.0,0.610179,0.0,0.002127,0.289301
inverse_multiquadric_rbf,0.000004,0.002431,0.610179,1.0,0.0,0.000245,0.050475
exponential_rbf,0.0,0.0,0.0,0.0,1.0,0.0,0.0
matern32_rbf,0.0,0.913152,0.002127,0.000245,0.0,1.0,0.144286
matern52_rbf,0.0,0.089894,0.289301,0.050475,0.0,0.144286,1.0


# Baltimore

In [12]:
df_mannwhit_balt = pd.DataFrame(columns=rbfn)
df_pval_balt = pd.DataFrame(columns=rbfn)
for i in rbfd:
    for j in rbfd:
        row1 = rbfd[i]['baltimore']
        row2 = rbfd[j]['baltimore']
        mannwhit, pval = stats.mannwhitneyu(row1, row2, method='auto')
        df_mannwhit_balt.loc[i, j] = mannwhit
        df_pval_balt.loc[i, j] = pval
        
df_mannwhit_balt

,original_rbf,squared_exponential_rbf,inverse_quadratic_rbf,inverse_multiquadric_rbf,exponential_rbf,matern32_rbf,matern52_rbf
original_rbf,1146098.0,1053288.0,1140972.0,1063075.0,951847.0,1095785.0,727761.0
squared_exponential_rbf,1001210.0,920724.5,1015943.0,950392.0,874430.0,961233.0,639808.0
inverse_quadratic_rbf,775752.0,702019.0,801378.0,765695.0,678562.0,735125.0,479960.0
inverse_multiquadric_rbf,688623.0,619657.0,699067.0,669324.5,570690.0,641449.0,414819.0
exponential_rbf,480397.0,409292.0,519074.0,523832.0,447458.0,415525.0,253280.0
matern32_rbf,863331.0,794725.0,903079.0,855709.0,808599.0,837218.0,562232.0
matern52_rbf,569737.0,523141.0,605002.0,576730.0,557442.0,546726.0,367224.5


In [13]:
df_pval_balt

,original_rbf,squared_exponential_rbf,inverse_quadratic_rbf,inverse_multiquadric_rbf,exponential_rbf,matern32_rbf,matern52_rbf
original_rbf,1.0,0.240294,0.0,0.0,0.0,0.0,0.000001
squared_exponential_rbf,0.240294,1.0,0.0,0.0,0.0,0.000024,0.000069
inverse_quadratic_rbf,0.0,0.0,1.0,0.052783,0.0,0.000007,0.000006
inverse_multiquadric_rbf,0.0,0.0,0.052783,1.0,0.090795,0.0,0.0
exponential_rbf,0.0,0.0,0.0,0.090795,1.0,0.0,0.0
matern32_rbf,0.0,0.000024,0.000007,0.0,0.0,1.0,0.582501
matern52_rbf,0.000001,0.000069,0.000006,0.0,0.0,0.582501,1.0


# Chester

In [14]:
df_mannwhit_ches = pd.DataFrame(columns=rbfn)
df_pval_ches = pd.DataFrame(columns=rbfn)
for i in rbfd:
    for j in rbfd:
        row1 = rbfd[i]['chester']
        row2 = rbfd[j]['chester']
        mannwhit, pval = stats.mannwhitneyu(row1, row2, method='auto')
        df_mannwhit_ches.loc[i, j] = mannwhit
        df_pval_ches.loc[i, j] = pval
        
df_mannwhit_ches

,original_rbf,squared_exponential_rbf,inverse_quadratic_rbf,inverse_multiquadric_rbf,exponential_rbf,matern32_rbf,matern52_rbf
original_rbf,1146098.0,1036064.0,1224148.0,900867.0,1170338.0,1362807.0,956131.0
squared_exponential_rbf,1018434.0,920724.5,1111756.0,806233.0,1074522.0,1254090.0,882888.0
inverse_quadratic_rbf,692576.0,606206.0,801378.0,578403.0,901128.0,921402.0,678491.0
inverse_multiquadric_rbf,850831.0,763816.0,886359.0,669324.5,850945.0,978745.0,688902.0
exponential_rbf,261906.0,209200.0,296508.0,243577.0,447458.0,333010.0,270027.0
matern32_rbf,596309.0,501868.0,716802.0,518413.0,891114.0,837218.0,643753.0
matern52_rbf,341367.0,280061.0,406471.0,302647.0,540695.0,465205.0,367224.5


In [15]:
df_pval_ches

,original_rbf,squared_exponential_rbf,inverse_quadratic_rbf,inverse_multiquadric_rbf,exponential_rbf,matern32_rbf,matern52_rbf
original_rbf,1.0,0.690995,0.0,0.205249,0.0,0.0,0.0
squared_exponential_rbf,0.690995,1.0,0.0,0.242349,0.0,0.0,0.0
inverse_quadratic_rbf,0.0,0.0,1.0,0.0,0.0,0.0,0.0
inverse_multiquadric_rbf,0.205249,0.242349,0.0,1.0,0.0,0.0,0.0
exponential_rbf,0.0,0.0,0.0,0.0,1.0,0.0,0.0
matern32_rbf,0.0,0.0,0.0,0.0,0.0,1.0,0.0
matern52_rbf,0.0,0.0,0.0,0.0,0.0,0.0,1.0


# Environment

In [17]:
df_mannwhit_env = pd.DataFrame(columns=rbfn)
df_pval_env = pd.DataFrame(columns=rbfn)
for i in rbfd:
    for j in rbfd:
        row1 = rbfd[i]['environment']
        row2 = rbfd[j]['environment']
        mannwhit, pval = stats.mannwhitneyu(row1, row2, method='auto')
        df_mannwhit_env.loc[i, j] = mannwhit
        df_pval_env.loc[i, j] = pval
        
df_mannwhit_env

,original_rbf,squared_exponential_rbf,inverse_quadratic_rbf,inverse_multiquadric_rbf,exponential_rbf,matern32_rbf,matern52_rbf
original_rbf,1146098.0,1026490.0,686789.0,486576.0,793653.0,1114141.0,683137.0
squared_exponential_rbf,1028008.0,920724.5,597161.0,423393.0,700065.0,989847.0,607375.0
inverse_quadratic_rbf,1229935.0,1120801.0,801378.0,506304.0,930378.0,1233437.0,759904.0
inverse_multiquadric_rbf,1265122.0,1146656.0,958458.0,669324.5,973701.0,1276452.0,797050.0
exponential_rbf,638591.0,583657.0,267258.0,120821.0,447458.0,636056.0,374166.0
matern32_rbf,844975.0,766111.0,404767.0,220706.0,588068.0,837218.0,497962.0
matern52_rbf,614361.0,555574.0,325058.0,194499.0,436556.0,610996.0,367224.5


In [18]:
df_pval_env

,original_rbf,squared_exponential_rbf,inverse_quadratic_rbf,inverse_multiquadric_rbf,exponential_rbf,matern32_rbf,matern52_rbf
original_rbf,1.0,0.972713,0.0,0.0,0.000006,0.0,0.031774
squared_exponential_rbf,0.972713,1.0,0.0,0.0,0.000209,0.0,0.077101
inverse_quadratic_rbf,0.0,0.0,1.0,0.0,0.0,0.0,0.0
inverse_multiquadric_rbf,0.0,0.0,0.0,1.0,0.0,0.0,0.0
exponential_rbf,0.000006,0.000209,0.0,0.0,1.0,0.112534,0.004719
matern32_rbf,0.0,0.0,0.0,0.0,0.112534,1.0,0.000061
matern52_rbf,0.031774,0.077101,0.0,0.0,0.004719,0.000061,1.0


# Recreation

In [19]:
df_mannwhit_rec = pd.DataFrame(columns=rbfn)
df_pval_rec = pd.DataFrame(columns=rbfn)
for i in rbfd:
    for j in rbfd:
        row1 = rbfd[i]['recreation']
        row2 = rbfd[j]['recreation']
        mannwhit, pval = stats.mannwhitneyu(row1, row2, method='auto')
        df_mannwhit_rec.loc[i, j] = mannwhit
        df_pval_rec.loc[i, j] = pval
        
df_mannwhit_rec

,original_rbf,squared_exponential_rbf,inverse_quadratic_rbf,inverse_multiquadric_rbf,exponential_rbf,matern32_rbf,matern52_rbf
original_rbf,1146098.0,1050358.5,1195092.0,869461.0,855692.0,1032961.5,680522.5
squared_exponential_rbf,1004139.5,920724.5,1054901.0,762311.0,754060.0,906691.5,597221.5
inverse_quadratic_rbf,721632.0,663061.0,801378.0,554739.5,565517.0,660003.0,434563.0
inverse_multiquadric_rbf,882237.0,807738.0,910022.5,669324.5,653112.5,792856.0,522593.0
exponential_rbf,576552.0,529662.0,632119.0,441409.5,447458.0,525830.0,346214.0
matern32_rbf,926154.5,849266.5,978201.0,704302.0,698294.0,837218.0,551467.5
matern52_rbf,616975.5,565727.5,650399.0,468956.0,464508.0,557490.5,367224.5


In [20]:
df_pval_rec

,original_rbf,squared_exponential_rbf,inverse_quadratic_rbf,inverse_multiquadric_rbf,exponential_rbf,matern32_rbf,matern52_rbf
original_rbf,1.0,0.120763,0.0,0.616937,0.0,0.000303,0.003542
squared_exponential_rbf,0.120763,1.0,0.0,0.059866,0.0,0.039446,0.125299
inverse_quadratic_rbf,0.0,0.0,1.0,0.0,0.009472,0.0,0.0
inverse_multiquadric_rbf,0.616937,0.059866,0.0,1.0,0.0,0.000229,0.002049
exponential_rbf,0.0,0.0,0.009472,0.0,1.0,0.0,0.0
matern32_rbf,0.000303,0.039446,0.0,0.000229,0.0,1.0,0.76798
matern52_rbf,0.003542,0.125299,0.0,0.002049,0.0,0.76798,1.0


In [ ]:
# create index
df = df_mannwhit_hyd.astype(float).round(2)
newcol = df.columns.to_list()
newcol = [x[:-4] for x in newcol]
df.columns=newcol
df.index = newcol        

In [35]:
df = df_mannwhit_rec.astype(float).round(1)
df2 = df_pval_rec.astype(float).round(2)
df.columns = newcol
df.index = newcol
df2.columns = newcol
df2.index = newcol
print("\\begin{table}[h]")
# print("\\parbox{.45\linewidth}{")
print("\\setlength{\\tabcolsep}{3.5pt}")
print("\\begin{tabular}{c | " + " | ".join(["c"] * len(df.columns)) + "}")
# print(" & ".join([str(x) for x in newcol]) + " \\\\")
print('& original & \makecell{squared\\\exponential} & \makecell{inverse\\\quadratic} & \makecell{inverse\\\multiquadric} & exponential & matern32 & matern52 \\\\')
print("\hline")
for i, row in df.iterrows():
    j = i.replace("_"," ")
    print(f"{j} & " + " & ".join([str(x) for x in row.values]) + " \\\\")
print("\\end{tabular}")
print("\\caption{Mann-Whitney test, recreation}")
print("\\end{table}")
print("")
# print("\\parbox{.45\linewidth}{")
# print("\\centering")
print("\\begin{table}[h]")
print("\\begin{tabular}{c | " + " | ".join(["c"] * len(df.columns)) + "}")
# print(" & " + " & ".join([str(x) for x in newcol]))
print("& original & \makecell{squared\\\exponential} & \makecell{inverse\\\quadratic} & \makecell{inverse\\\multiquadric} & exponential & matern32 & matern52 \\\\")
print("\hline")
for i, row in df2.iterrows():
    j = i.replace("_"," ")
    print(f"{j} & " + " & ".join([str(x) for x in row.values]) + " \\\\")
print("\\end{tabular}")
print("\\caption{Mann-Whitney test, P value, recreation}")
# print("}")
print("\\end{table}")

\begin{table}[h]
\setlength{\tabcolsep}{3.5pt}
\begin{tabular}{c | c | c | c | c | c | c | c}
& original & \makecell{squared\\exponential} & \makecell{inverse\\quadratic} & \makecell{inverse\\multiquadric} & exponential & matern32 & matern52 \\
\hline
original & 1146098.0 & 1050358.5 & 1195092.0 & 869461.0 & 855692.0 & 1032961.5 & 680522.5 \\
squared exponential & 1004139.5 & 920724.5 & 1054901.0 & 762311.0 & 754060.0 & 906691.5 & 597221.5 \\
inverse quadratic & 721632.0 & 663061.0 & 801378.0 & 554739.5 & 565517.0 & 660003.0 & 434563.0 \\
inverse multiquadric & 882237.0 & 807738.0 & 910022.5 & 669324.5 & 653112.5 & 792856.0 & 522593.0 \\
exponential & 576552.0 & 529662.0 & 632119.0 & 441409.5 & 447458.0 & 525830.0 & 346214.0 \\
matern32 & 926154.5 & 849266.5 & 978201.0 & 704302.0 & 698294.0 & 837218.0 & 551467.5 \\
matern52 & 616975.5 & 565727.5 & 650399.0 & 468956.0 & 464508.0 & 557490.5 & 367224.5 \\
\end{tabular}
\caption{Mann-Whitney test, recreation}
\end{table}

\begin{table}[h]
